<a href="https://colab.research.google.com/github/VGandhi27/Depression-Detection/blob/main/Copy_of_Untitled.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as py
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
import seaborn as sns
import string
import nltk

In [2]:
df=pd.read_csv('13100607.csv')
# df_meta=pd.read_csv('13100607_MetaData.csv')

In [3]:
df.head()

,REF_DATE,GEO,DGUID,Age group,Sex,Probability of depression,Characteristics,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,2003,Newfoundland and Labrador,10,"Total, 12 years and over",Both sexes,Total population for the variable probability ...,Number of persons,Persons,249,units,0,v26656800,2.1.1.1.1,460130.0,NaN,NaN,NaN,0
1,2003,Newfoundland and Labrador,10,"Total, 12 years and over",Both sexes,Total population for the variable probability ...,"Low 95% confidence interval, number of persons",Persons,249,units,0,v26656801,2.1.1.1.2,460130.0,NaN,NaN,NaN,0
2,2003,Newfoundland and Labrador,10,"Total, 12 years and over",Both sexes,Total population for the variable probability ...,"High 95% confidence interval, number of persons",Persons,249,units,0,v26656802,2.1.1.1.3,460130.0,NaN,NaN,NaN,0
3,2003,Newfoundland and Labrador,10,"Total, 12 years and over",Both sexes,Total population for the variable probability ...,Coefficient of variation for number of persons,Persons,249,units,0,v26656803,2.1.1.1.4,0.0,NaN,NaN,NaN,1
4,2003,Newfoundland and Labrador,10,"Total, 12 years and over",Both sexes,Total population for the variable probability ...,Percent,Percent,239,units,0,v26656804,2.1.1.1.5,100.0,NaN,NaN,NaN,1


In [4]:
# df_meta.head()

In [5]:
df.shape

(90048, 18)

In [6]:
# df_meta.shape

In [7]:
df.isnull().sum()

REF_DATE                         0
GEO                              0
DGUID                            0
Age group                        0
Sex                              0
Probability of depression        0
Characteristics                  0
UOM                              0
UOM_ID                           0
SCALAR_FACTOR                    0
SCALAR_ID                        0
VECTOR                           0
COORDINATE                       0
VALUE                        36044
STATUS                       42872
SYMBOL                       90048
TERMINATED                   90048
DECIMALS                         0
dtype: int64

In [8]:
df=df.drop(columns=['VALUE','STATUS','SYMBOL','TERMINATED'] ,axis=1)

In [9]:
# df=df.dropna(inplace=True)
df.duplicated().sum()

0

In [10]:
df.head()

,REF_DATE,GEO,DGUID,Age group,Sex,Probability of depression,Characteristics,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,DECIMALS
0,2003,Newfoundland and Labrador,10,"Total, 12 years and over",Both sexes,Total population for the variable probability ...,Number of persons,Persons,249,units,0,v26656800,2.1.1.1.1,0
1,2003,Newfoundland and Labrador,10,"Total, 12 years and over",Both sexes,Total population for the variable probability ...,"Low 95% confidence interval, number of persons",Persons,249,units,0,v26656801,2.1.1.1.2,0
2,2003,Newfoundland and Labrador,10,"Total, 12 years and over",Both sexes,Total population for the variable probability ...,"High 95% confidence interval, number of persons",Persons,249,units,0,v26656802,2.1.1.1.3,0
3,2003,Newfoundland and Labrador,10,"Total, 12 years and over",Both sexes,Total population for the variable probability ...,Coefficient of variation for number of persons,Persons,249,units,0,v26656803,2.1.1.1.4,1
4,2003,Newfoundland and Labrador,10,"Total, 12 years and over",Both sexes,Total population for the variable probability ...,Percent,Percent,239,units,0,v26656804,2.1.1.1.5,1


In [11]:
df['Probability of depression'].value_counts()

Total population for the variable probability of depression    22512
Probability of depression, less than 0.9                       22512
Probability of depression, 0.9 or greater                      22512
Probability of depression, not stated                          22512
Name: Probability of depression, dtype: int64

In [12]:
df.groupby('Probability of depression').mean()

C:\Users\Vidushi\AppData\Local\Temp\ipykernel_6368\1608917492.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby('Probability of depression').mean()


,REF_DATE,DGUID,UOM_ID,SCALAR_ID,DECIMALS
Probability of depression,,,,,
"Probability of depression, 0.9 or greater",2003.0,2904.671642,244.0,0.0,0.625
"Probability of depression, less than 0.9",2003.0,2904.671642,244.0,0.0,0.625
"Probability of depression, not stated",2003.0,2904.671642,244.0,0.0,0.625
Total population for the variable probability of depression,2003.0,2904.671642,244.0,0.0,0.625


In [16]:
def remove_punctuation(text):
    no_punct=[words for words in text if words not in string.punctuation]
    words_wo_punct=''.join(no_punct)
    return words_wo_punct
df['clean_Probability_of_depression']=df['Probability of depression'].apply(lambda x: remove_punctuation(x))
df['clean_Age_Group']=df['Age group'].apply(lambda x: remove_punctuation(x))
df['clean_Characteristics']=df['Characteristics'].apply(lambda x: remove_punctuation(x))
df.head()

,REF_DATE,GEO,DGUID,Age group,Sex,Probability of depression,Characteristics,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,DECIMALS,clean_Probability_of_depression,clean_Age_Group,clean_Characteristics
0,2003,Newfoundland and Labrador,10,"Total, 12 years and over",Both sexes,Total population for the variable probability ...,Number of persons,Persons,249,units,0,v26656800,2.1.1.1.1,0,Total population for the variable probability ...,Total 12 years and over,Number of persons
1,2003,Newfoundland and Labrador,10,"Total, 12 years and over",Both sexes,Total population for the variable probability ...,"Low 95% confidence interval, number of persons",Persons,249,units,0,v26656801,2.1.1.1.2,0,Total population for the variable probability ...,Total 12 years and over,Low 95 confidence interval number of persons
2,2003,Newfoundland and Labrador,10,"Total, 12 years and over",Both sexes,Total population for the variable probability ...,"High 95% confidence interval, number of persons",Persons,249,units,0,v26656802,2.1.1.1.3,0,Total population for the variable probability ...,Total 12 years and over,High 95 confidence interval number of persons
3,2003,Newfoundland and Labrador,10,"Total, 12 years and over",Both sexes,Total population for the variable probability ...,Coefficient of variation for number of persons,Persons,249,units,0,v26656803,2.1.1.1.4,1,Total population for the variable probability ...,Total 12 years and over,Coefficient of variation for number of persons
4,2003,Newfoundland and Labrador,10,"Total, 12 years and over",Both sexes,Total population for the variable probability ...,Percent,Percent,239,units,0,v26656804,2.1.1.1.5,1,Total population for the variable probability ...,Total 12 years and over,Percent


In [20]:
import re
#convert all words to lowercase
df['clean_Probability_of_depression'] = df['clean_Probability_of_depression'].str.lower()
df['clean_Age_Group'] = df['clean_Age_Group'].str.lower()
df['clean_Characteristics'] = df['clean_Characteristics'].str.lower()
df['clean_UOM'] = df['UOM'].str.lower()
df['clean_Sex'] = df['Sex'].str.lower()
df['clean_GEO'] = df['GEO'].str.lower()
#remove emoji
df['clean_Characteristics'] = df['clean_Characteristics'].str.replace('[^\w\s#@/:%.,_-]', '', flags=re.UNICODE)

df.head()

C:\Users\Vidushi\AppData\Local\Temp\ipykernel_6368\3516803623.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  df['clean_Characteristics'] = df['clean_Characteristics'].str.replace('[^\w\s#@/:%.,_-]', '', flags=re.UNICODE)


,REF_DATE,GEO,DGUID,Age group,Sex,Probability of depression,Characteristics,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,DECIMALS,clean_Probability_of_depression,clean_Age_Group,clean_Characteristics,clean_UOM,clean_Sex,clean_GEO
0,2003,Newfoundland and Labrador,10,"Total, 12 years and over",Both sexes,Total population for the variable probability ...,Number of persons,Persons,249,units,0,v26656800,2.1.1.1.1,0,total population for the variable probability ...,total 12 years and over,number of persons,persons,both sexes,newfoundland and labrador
1,2003,Newfoundland and Labrador,10,"Total, 12 years and over",Both sexes,Total population for the variable probability ...,"Low 95% confidence interval, number of persons",Persons,249,units,0,v26656801,2.1.1.1.2,0,total population for the variable probability ...,total 12 years and over,low 95 confidence interval number of persons,persons,both sexes,newfoundland and labrador
2,2003,Newfoundland and Labrador,10,"Total, 12 years and over",Both sexes,Total population for the variable probability ...,"High 95% confidence interval, number of persons",Persons,249,units,0,v26656802,2.1.1.1.3,0,total population for the variable probability ...,total 12 years and over,high 95 confidence interval number of persons,persons,both sexes,newfoundland and labrador
3,2003,Newfoundland and Labrador,10,"Total, 12 years and over",Both sexes,Total population for the variable probability ...,Coefficient of variation for number of persons,Persons,249,units,0,v26656803,2.1.1.1.4,1,total population for the variable probability ...,total 12 years and over,coefficient of variation for number of persons,persons,both sexes,newfoundland and labrador
4,2003,Newfoundland and Labrador,10,"Total, 12 years and over",Both sexes,Total population for the variable probability ...,Percent,Percent,239,units,0,v26656804,2.1.1.1.5,1,total population for the variable probability ...,total 12 years and over,percent,percent,both sexes,newfoundland and labrador


In [24]:
df.keys()

Index(['GEO', 'DGUID', 'Age group', 'Sex', 'Probability of depression',
       'Characteristics', 'UOM', 'UOM_ID', 'clean_Probability_of_depression',
       'clean_Age_Group', 'clean_Characteristics', 'clean_UOM', 'clean_Sex',
       'clean_GEO'],
      dtype='object')

In [22]:
df=df.drop(columns=['REF_DATE','SCALAR_FACTOR','SCALAR_ID', 'VECTOR', 'COORDINATE', 'DECIMALS'],axis=1)

In [23]:
df.head()

,GEO,DGUID,Age group,Sex,Probability of depression,Characteristics,UOM,UOM_ID,clean_Probability_of_depression,clean_Age_Group,clean_Characteristics,clean_UOM,clean_Sex,clean_GEO
0,Newfoundland and Labrador,10,"Total, 12 years and over",Both sexes,Total population for the variable probability ...,Number of persons,Persons,249,total population for the variable probability ...,total 12 years and over,number of persons,persons,both sexes,newfoundland and labrador
1,Newfoundland and Labrador,10,"Total, 12 years and over",Both sexes,Total population for the variable probability ...,"Low 95% confidence interval, number of persons",Persons,249,total population for the variable probability ...,total 12 years and over,low 95 confidence interval number of persons,persons,both sexes,newfoundland and labrador
2,Newfoundland and Labrador,10,"Total, 12 years and over",Both sexes,Total population for the variable probability ...,"High 95% confidence interval, number of persons",Persons,249,total population for the variable probability ...,total 12 years and over,high 95 confidence interval number of persons,persons,both sexes,newfoundland and labrador
3,Newfoundland and Labrador,10,"Total, 12 years and over",Both sexes,Total population for the variable probability ...,Coefficient of variation for number of persons,Persons,249,total population for the variable probability ...,total 12 years and over,coefficient of variation for number of persons,persons,both sexes,newfoundland and labrador
4,Newfoundland and Labrador,10,"Total, 12 years and over",Both sexes,Total population for the variable probability ...,Percent,Percent,239,total population for the variable probability ...,total 12 years and over,percent,percent,both sexes,newfoundland and labrador


In [25]:
#tokenization
nltk.download('punkt')
def tokenize(text):
    split=re.split("\W+",text) 
    return split
df['clean_Probability_of_depression_tokenize']=df['clean_Probability_of_depression'].apply(lambda x: tokenize(x.lower()))
df['clean_Age_Group_tokenize']=df['clean_Age_Group'].apply(lambda x: tokenize(x.lower()))
df['clean_Characteristics_tokenize']=df['clean_Characteristics'].apply(lambda x: tokenize(x.lower()))
df['clean_UOM_tokenize']=df['clean_UOM'].apply(lambda x: tokenize(x.lower()))
df['clean_UOM_tokenize']=df['clean_UOM'].apply(lambda x: tokenize(x.lower()))
df['clean_Sex_tokenize']=df['clean_Sex'].apply(lambda x: tokenize(x.lower()))
df['clean_GEO_tokenize']=df['clean_GEO'].apply(lambda x: tokenize(x.lower()))

df.head()

[nltk_data] Error loading punkt: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


,GEO,DGUID,Age group,Sex,Probability of depression,Characteristics,UOM,UOM_ID,clean_Probability_of_depression,clean_Age_Group,clean_Characteristics,clean_UOM,clean_Sex,clean_GEO,clean_Probability_of_depression_tokenize,clean_Age_Group_tokenize,clean_Characteristics_tokenize,clean_UOM_tokenize,clean_Sex_tokenize,clean_GEO_tokenize
0,Newfoundland and Labrador,10,"Total, 12 years and over",Both sexes,Total population for the variable probability ...,Number of persons,Persons,249,total population for the variable probability ...,total 12 years and over,number of persons,persons,both sexes,newfoundland and labrador,"[total, population, for, the, variable, probab...","[total, 12, years, and, over]","[number, of, persons]",[persons],"[both, sexes]","[newfoundland, and, labrador]"
1,Newfoundland and Labrador,10,"Total, 12 years and over",Both sexes,Total population for the variable probability ...,"Low 95% confidence interval, number of persons",Persons,249,total population for the variable probability ...,total 12 years and over,low 95 confidence interval number of persons,persons,both sexes,newfoundland and labrador,"[total, population, for, the, variable, probab...","[total, 12, years, and, over]","[low, 95, confidence, interval, number, of, pe...",[persons],"[both, sexes]","[newfoundland, and, labrador]"
2,Newfoundland and Labrador,10,"Total, 12 years and over",Both sexes,Total population for the variable probability ...,"High 95% confidence interval, number of persons",Persons,249,total population for the variable probability ...,total 12 years and over,high 95 confidence interval number of persons,persons,both sexes,newfoundland and labrador,"[total, population, for, the, variable, probab...","[total, 12, years, and, over]","[high, 95, confidence, interval, number, of, p...",[persons],"[both, sexes]","[newfoundland, and, labrador]"
3,Newfoundland and Labrador,10,"Total, 12 years and over",Both sexes,Total population for the variable probability ...,Coefficient of variation for number of persons,Persons,249,total population for the variable probability ...,total 12 years and over,coefficient of variation for number of persons,persons,both sexes,newfoundland and labrador,"[total, population, for, the, variable, probab...","[total, 12, years, and, over]","[coefficient, of, variation, for, number, of, ...",[persons],"[both, sexes]","[newfoundland, and, labrador]"
4,Newfoundland and Labrador,10,"Total, 12 years and over",Both sexes,Total population for the variable probability ...,Percent,Percent,239,total population for the variable probability ...,total 12 years and over,percent,percent,both sexes,newfoundland and labrador,"[total, population, for, the, variable, probab...","[total, 12, years, and, over]",[percent],[percent],"[both, sexes]","[newfoundland, and, labrador]"


In [27]:
#stopwords
import nltk
nltk.download('stopwords')
stopword = nltk.corpus.stopwords.words('english')
def remove_stopwords(text):
    text=[word for word in text if word not in stopword]
    return text
df['clean_Probability_of_depression_tokenize_stopwords'] = df['clean_Probability_of_depression_tokenize'].apply(lambda x: remove_stopwords(x))
df['clean_Age_Group_stopwords'] = df['clean_Age_Group_tokenize'].apply(lambda x: remove_stopwords(x))
df['clean_Characteristics_tokenize_stopwords'] = df['clean_Characteristics_tokenize'].apply(lambda x: remove_stopwords(x))
df['clean_Sex_tokenize_stopwords'] = df['clean_Sex_tokenize'].apply(lambda x: remove_stopwords(x))
df['clean_GEO_tokenize_stopwords'] = df['clean_GEO_tokenize'].apply(lambda x: remove_stopwords(x))
df['clean_UOM_tokenize_stopwords'] = df['clean_UOM_tokenize'].apply(lambda x: remove_stopwords(x))


df.head(10)

[nltk_data] Error loading stopwords: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - 'C:\\Users\\Vidushi/nltk_data'
    - 'C:\\Users\\Vidushi\\AppData\\Local\\Programs\\Python\\Python311\\nltk_data'
    - 'C:\\Users\\Vidushi\\AppData\\Local\\Programs\\Python\\Python311\\share\\nltk_data'
    - 'C:\\Users\\Vidushi\\AppData\\Local\\Programs\\Python\\Python311\\lib\\nltk_data'
    - 'C:\\Users\\Vidushi\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [ ]:
x=df.drop(columns='Probability of depression',axis=1)
y=df['Probability of depression']

In [ ]:
print(x)

       REF_DATE                        GEO  DGUID                 Age group  \
0          2003  Newfoundland and Labrador     10  Total, 12 years and over   
1          2003  Newfoundland and Labrador     10  Total, 12 years and over   
2          2003  Newfoundland and Labrador     10  Total, 12 years and over   
3          2003  Newfoundland and Labrador     10  Total, 12 years and over   
4          2003  Newfoundland and Labrador     10  Total, 12 years and over   
...         ...                        ...    ...                       ...   
90043      2003                    Nunavut   6201         75 years and over   
90044      2003                    Nunavut   6201         75 years and over   
90045      2003                    Nunavut   6201         75 years and over   
90046      2003                    Nunavut   6201         75 years and over   
90047      2003                    Nunavut   6201         75 years and over   

              Sex                                  

In [ ]:
print(y)

0        Total population for the variable probability ...
1        Total population for the variable probability ...
2        Total population for the variable probability ...
3        Total population for the variable probability ...
4        Total population for the variable probability ...
                               ...                        
90043                Probability of depression, not stated
90044                Probability of depression, not stated
90045                Probability of depression, not stated
90046                Probability of depression, not stated
90047                Probability of depression, not stated
Name: Probability of depression, Length: 90048, dtype: object


In [ ]:
x.shape

(90048, 13)

In [ ]:
y.shape

(90048,)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.5, random_state=0)

In [ ]:
print(x.shape,X_test.shape,X_train.shape)

(90048, 13) (45024, 13) (45024, 13)


In [ ]:
print(y_train)

32739    Total population for the variable probability ...
87596             Probability of depression, less than 0.9
65450             Probability of depression, less than 0.9
36851            Probability of depression, 0.9 or greater
39037                Probability of depression, not stated
                               ...                        
21243                Probability of depression, not stated
45891    Total population for the variable probability ...
42613            Probability of depression, 0.9 or greater
43567             Probability of depression, less than 0.9
68268             Probability of depression, less than 0.9
Name: Probability of depression, Length: 45024, dtype: object


In [ ]:
df.head()

,REF_DATE,GEO,DGUID,Age group,Sex,Probability of depression,Characteristics,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,DECIMALS
0,2003,Newfoundland and Labrador,10,"Total, 12 years and over",Both sexes,Total population for the variable probability ...,Number of persons,Persons,249,units,0,v26656800,2.1.1.1.1,0
1,2003,Newfoundland and Labrador,10,"Total, 12 years and over",Both sexes,Total population for the variable probability ...,"Low 95% confidence interval, number of persons",Persons,249,units,0,v26656801,2.1.1.1.2,0
2,2003,Newfoundland and Labrador,10,"Total, 12 years and over",Both sexes,Total population for the variable probability ...,"High 95% confidence interval, number of persons",Persons,249,units,0,v26656802,2.1.1.1.3,0
3,2003,Newfoundland and Labrador,10,"Total, 12 years and over",Both sexes,Total population for the variable probability ...,Coefficient of variation for number of persons,Persons,249,units,0,v26656803,2.1.1.1.4,1
4,2003,Newfoundland and Labrador,10,"Total, 12 years and over",Both sexes,Total population for the variable probability ...,Percent,Percent,239,units,0,v26656804,2.1.1.1.5,1


In [ ]:
df.target_names

AttributeError: ignored

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline


In [ ]:
model=make_pipeline(TfidfVectorizer(),MultinomialNB())

In [ ]:
model.fit(X_train, y_train)

ValueError: ignored

In [ ]:
model = GaussianNB() 

In [ ]:
# y_pred = model.fit(X_train, y_train).predict(X_test)

In [ ]:
X_train.keys()

Index(['REF_DATE', 'GEO', 'DGUID', 'Age group', 'Sex', 'Characteristics',
       'UOM', 'UOM_ID', 'SCALAR_FACTOR', 'SCALAR_ID', 'VECTOR', 'COORDINATE',
       'DECIMALS'],
      dtype='object')

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline

model = make_pipeline(TfidfVectorizer(analyzer = "word", ngram_range=(1,3)), MultinomialNB(alpha = 10))

In [ ]:
model.fit(X_train,y_train)

ValueError: Found input variables with inconsistent numbers of samples: [13, 45024]